<a href="https://colab.research.google.com/github/Leon-web-net/Learning_ML/blob/main/Ensemble_Methods/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openml

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import openml

# ID = 31 # credit-g (German Credit dataset)
# dataset = openml.datasets.get_dataset(ID)

# df, *_ = dataset.get_data(dataset_format="dataframe")


# save_path = "./drive/MyDrive/datasets/credit-g.csv"
# df.to_csv(save_path, index=False)
# print("File saved as csv to ", save_path)

# print(df.head())
# print("Shape:", df.shape)

In [ ]:
dataset_path = "./drive/MyDrive/datasets/credit-g.csv"
df = pd.read_csv(dataset_path)
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951.0,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096.0,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870.0,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [ ]:
df.columns

Index(['checking_status', 'duration', 'credit_history', 'purpose',
       'credit_amount', 'savings_status', 'employment',
       'installment_commitment', 'personal_status', 'other_parties',
       'residence_since', 'property_magnitude', 'age', 'other_payment_plans',
       'housing', 'existing_credits', 'job', 'num_dependents', 'own_telephone',
       'foreign_worker', 'class'],
      dtype='object')

# Preprocessing

- About 70% good / 30% bad → not crazy imbalanced, but enough that if you always guessed “good” you’d already get 70% accuracy.

That’s why looking beyond accuracy (to precision, recall, F1) is important.

In [ ]:
df["class"].value_counts(),df["class"].value_counts(normalize=True)

(class
 good    700
 bad     300
 Name: count, dtype: int64,
 class
 good    0.7
 bad     0.3
 Name: proportion, dtype: float64)

In [ ]:
X = df.drop(columns=["class"])
y = df["class"]

X.shape,y.shape

((1000, 20), (1000,))

One hot encoding for Linear regression can be used for Trees aswell

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status_<0,checking_status_>=200,checking_status_no checking,...,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_skilled,job_unemp/unskilled non res,job_unskilled resident,own_telephone_yes,foreign_worker_yes
0,6,1169.0,4,4,67,2,1,True,False,False,...,True,True,False,True,False,True,False,False,True,True
1,48,5951.0,2,2,22,1,1,False,False,False,...,True,True,False,True,False,True,False,False,False,True
2,12,2096.0,2,3,49,1,2,False,False,True,...,True,True,False,True,False,False,False,True,False,True
3,42,7882.0,2,4,45,1,2,True,False,False,...,False,True,False,False,False,True,False,False,False,True
4,24,4870.0,3,4,53,2,2,True,False,False,...,False,True,False,False,False,True,False,False,False,True


In [ ]:
X_encoded.shape

(1000, 48)

Integer encoding for tree models

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)  # "good"/"bad" → 1/0 (or 0/1)
y_encoded[:10], y[:10]

(array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0]),
 0    good
 1     bad
 2    good
 3    good
 4     bad
 5    good
 6    good
 7    good
 8    good
 9     bad
 Name: class, dtype: object)

In [ ]:
X_encoded = X.copy()

cats_cols = X.select_dtypes(include=["object","category"]).columns

le_dict = {}
for col in cats_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col]).astype(int)
    le_dict[col] = le

X_encoded.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,1,6,1,6,1169.0,4,3,4,3,2,4,3,67,1,1,2,1,1,1,1
1,0,48,3,6,5951.0,2,0,2,0,2,2,3,22,1,1,1,1,1,0,1
2,3,12,1,2,2096.0,2,1,2,3,2,3,3,49,1,1,1,3,2,0,1
3,1,42,3,3,7882.0,2,1,2,3,1,4,1,45,1,0,1,1,2,0,1
4,1,24,2,4,4870.0,2,0,3,3,2,4,2,53,1,0,2,1,2,0,1


In [ ]:
X_encoded.dtypes

,0
checking_status,int64
duration,int64
credit_history,int64
purpose,int64
credit_amount,float64
savings_status,int64
employment,int64
installment_commitment,int64
personal_status,int64
other_parties,int64


In [ ]:
X_encoded['checking_status'].unique()

array([1, 0, 3, 2])

In [ ]:
encoded_vals = X_encoded['checking_status'].head()

# Decode back to original categories
original_vals = le_dict['checking_status'].inverse_transform(encoded_vals)
print(original_vals)

['<0' '0<=X<200' 'no checking' '<0' '<0']


In [ ]:
unique,counts = np.unique(y_encoded,return_counts=True)
dict(zip(unique,counts))

{np.int64(0): np.int64(300), np.int64(1): np.int64(700)}

# Decision Trees

In [ ]:
class TreeNode:
  def __init__(self,feature=None, threshold=None, left=None, right=None,*,value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

In [ ]:
### Alternative to this is entropy
def gini(y):
  classes = np.unique(y)
  m = len(y)
  if m == 0:
    return 0
  return 1 - sum((np.sum(y==c)/m)**2 for c in classes)


### 🌳 Best Split in Decision Trees
<details>

At each node, the algorithm tries **all features** and **all possible thresholds** to find the split that minimizes impurity.  

---

**Candidate Split**  
For a given feature index *f* and threshold *t*:  

$$
X_{\text{left}} = \{x : x[f] \leq t\}, \quad
X_{\text{right}} = \{x : x[f] > t\}
$$  

This divides the data into **left** and **right** subsets.

---

**Weighted Impurity of the Split**  
If $G(\cdot)$ is the impurity function (e.g. Gini, Entropy), the split impurity is the weighted average:  

$$
G_{\text{split}} = \frac{|L|}{m} \, G(L) + \frac{|R|}{m} \, G(R)
$$  

- $m$: total samples at the node  
- $|L|$: number of samples in left split  
- $|R|$: number of samples in right split  
- $G(L), G(R)$: impurity of left and right subsets  

---

**Goal**  
The algorithm chooses the feature $f^*$ and threshold $t^*$ that minimize:  

$$
(f^*, t^*) = \arg\min_{f,t} \; G_{\text{split}}(f,t)
$$  

✅ Intuition: lower impurity = groups are more “pure” (closer to a single class).


In [ ]:
def best_split(X,y):
  m,n = X.shape
  best_gini = 1.0
  split_idx, split_thresh = None, None

  for feature_idx in range(n):
    thresholds = np.unique(X[:,feature_idx])
    for thresh in thresholds:
      left_mask = X[:,feature_idx] <= thresh
      right_mask = ~left_mask
      y_left, y_right = y[left_mask], y[right_mask]

      g = (len(y_left)*gini(y_left) + len(y_right)*gini(y_right))/m

      if g < best_gini:
        best_gini = g
        split_idx = feature_idx
        split_thresh = thresh

  return split_idx, split_thresh


  ### improve split by using midpoints for thresholds


## 🌳 Build Tree (Decision Tree Construction)

<details>
The tree is built recursively using **Top-Down Induction of Decision Trees (TDIDT)**.

1. **Find the best split** at the current node:
   - Choose feature and threshold that minimize the weighted impurity:
     
     $$
     G_{\text{split}} \;=\; \frac{|L|}{m} G(L) \;+\; \frac{|R|}{m} G(R)
     $$

     where  

     $$
     L = \{x : x[\text{feature}] \leq \text{threshold}\}, \quad
     R = \{x : x[\text{feature}] > \text{threshold}\}
     $$

2. **Split the data** into left child $L$ and right child $R$.

3. **Recursively build subtrees** for left and right children.

4. **Stopping criteria** (make leaf node):
   - No split improves impurity  
   - Max depth reached  
   - Node is pure (all same class)  
   - Too few samples remain  

---

### 🔑 Traversal
- **Pre-order traversal**:  
  Root → Left → Right  
  (process current node, then build left child, then right child)

This recursive process is called:

$$
\textbf{Recursive Top-Down Induction of Decision Trees (TDIDT)}
$$


In [ ]:


def build_tree(X, y, depth=0, max_depth=5):
    n_samples, n_features = X.shape

    # --- safeguard: if no samples, return default leaf ---
    if n_samples == 0:
        return TreeNode(value=0)   # default class 0 (could also use majority from parent)

    # stopping conditions (pure node or depth reached)
    if depth >= max_depth or len(set(y)) == 1:
        counts = np.bincount(y)
        return TreeNode(value=np.argmax(counts))

    # find best split
    feature, threshold = best_split(X, y)

    # safeguard: if no valid split found
    if feature is None:
        counts = np.bincount(y)
        return TreeNode(value=np.argmax(counts))

    # split data
    left_mask = X[:, feature] <= threshold
    right_mask = ~left_mask

    # safeguard: if split fails (all left or all right)
    if left_mask.sum() == 0 or right_mask.sum() == 0:
        counts = np.bincount(y)
        return TreeNode(value=np.argmax(counts))

    # recurse
    left_child = build_tree(X[left_mask], y[left_mask], depth+1, max_depth)
    right_child = build_tree(X[right_mask], y[right_mask], depth+1, max_depth)

    return TreeNode(feature, threshold, left_child, right_child)


In [ ]:
def predict_tree(node,x):
  if node.value is not None:
    return node.value

  if x[node.feature] <= node.threshold:
    return predict_tree(node.left,x)
  else:
    return predict_tree(node.right,x)

def predict_tree_batch(node,X):
  return np.array([predict_tree(node,x) for x in X])

In [ ]:
from sklearn.model_selection import train_test_split


# Convert X_encoded and y_encoded to numpy arrays
X_np = X_encoded.to_numpy()
y_np = y_encoded

X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

In [ ]:
MAX_DEPTH = 5

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

tree = build_tree(X_train, y_train, max_depth=MAX_DEPTH)
y_train_pred = predict_tree_batch(tree, X_train)
y_test_pred = predict_tree_batch(tree, X_test)


print("Manual Tree Performance:")
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy: ", accuracy_score(y_test, y_test_pred))
print("\nTest Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

Manual Tree Performance:
Train Accuracy: 0.805
Test Accuracy:  0.745

Test Confusion Matrix:
 [[ 24  35]
 [ 16 125]]

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.41      0.48        59
           1       0.78      0.89      0.83       141

    accuracy                           0.74       200
   macro avg       0.69      0.65      0.66       200
weighted avg       0.73      0.74      0.73       200



In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create sklearn tree with similar max_depth
sk_tree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=42)
sk_tree.fit(X_train, y_train)

y_train_sk = sk_tree.predict(X_train)
y_test_sk = sk_tree.predict(X_test)

print("Sklearn Decision Tree Performance:")
print("Train Accuracy:", accuracy_score(y_train, y_train_sk))
print("Test Accuracy: ", accuracy_score(y_test, y_test_sk))
print("\nTest Confusion Matrix:\n", confusion_matrix(y_test, y_test_sk))
print("\nClassification Report:\n", classification_report(y_test, y_test_sk))


Sklearn Decision Tree Performance:
Train Accuracy: 0.805
Test Accuracy:  0.745

Test Confusion Matrix:
 [[ 23  36]
 [ 15 126]]

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.39      0.47        59
           1       0.78      0.89      0.83       141

    accuracy                           0.74       200
   macro avg       0.69      0.64      0.65       200
weighted avg       0.73      0.74      0.73       200



## Why Plain Decision Trees Struggle on German Credit Dataset

<details>

### 1️⃣ Limitations of Single Decision Trees
- **High variance / overfitting:** Deep trees memorize training data but generalize poorly.
- **Axis-aligned splits only:** Complex interactions between features require many splits.
- **Sensitive to small data changes:** Small datasets can lead to unstable splits.
- **Class imbalance:** Minority class (bad credit) often has lower recall.

### 2️⃣ Observations from Current Results
- Train Accuracy ~ 0.805, Test Accuracy ~ 0.745
- Low recall for class 0 (bad credit): ~0.39–0.41
- High recall for class 1 (good credit): ~0.83–0.89
- Increasing depth improves train accuracy but causes **overfitting**, test accuracy may drop.

### 3️⃣ Alternative Methods for Better Performance
1. **Ensemble Methods**
   - **Random Forests:** Bagging + averaging reduces variance.
   - **Gradient Boosting / XGBoost / LightGBM / CatBoost:** Sequential trees correct previous errors; strong on tabular data.

2. **Feature Engineering**
   - Add interaction terms, polynomial features, or engineered features to capture patterns.

3. **Hybrid / Stacking Approaches**
   - Combine trees, linear models, or other algorithms to improve robustness.

### ✅ Recommendation
- Use **Random Forests** or **Gradient Boosted Trees** next.  
- Tune hyperparameters (`max_depth`, `n_estimators`, `min_samples_leaf`).  
- Consider class weighting or SMOTE to handle minority class better.


# Random Forest

<details>

## Idea
Random Forest = an **ensemble of decision trees**.  
It reduces overfitting by combining many weak learners (trees) into a strong learner.  
Predictions are made by **majority vote** (classification) or **average** (regression).

---

## Key Steps

1. **Bootstrap Sampling**
   - For each tree, take a random sample (with replacement) of the training data.
   - Each tree sees a slightly different dataset.

2. **Feature Bagging**
   - At each split, only consider a random subset of features (not all features).
   - Typical choice:  
     - Classification → √(number of features)  
     - Regression → (number of features) / 3

3. **Tree Growth**
   - Each tree is trained on its bootstrap sample and feature subset.
   - Trees are often grown **deep** (high variance individually).

4. **Aggregation**
   - For classification: majority vote across all trees.
   - For regression: average the predictions.

---

## Why It Works
- **Reduces variance** compared to a single decision tree.  
- Decorrelated trees (via bootstrap + feature bagging) prevent them from making the same mistakes.  
- Handles **tabular data** with mixed numerical/categorical features very well.

---

## Formula for Split (per tree)
For a feature `f` and threshold `t`:


$L = \{x : x[f] \leq t\}, \quad R = \{x : x[f] > t\}$

Weighted impurity:

$
G_{\text{split}} = \frac{|L|}{m} G(L) + \frac{|R|}{m} G(R)
$

Each tree chooses the best split for its sampled data/features.

---

## Final Prediction

$
\hat{y} = \text{mode}\{ \hat{y}^{(1)}, \hat{y}^{(2)}, \dots, \hat{y}^{(T)} \}
$

where \(T\) = number of trees.

In [ ]:
class RandomForest:
  def __init__(self,n_trees=10,max_depth=5,n_feats=None):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.trees = []
    self.feat_idxs = []

  def fit(self,X,y):
    self.trees = []
    n_samples, n_features = X.shape
    self.n_feats = self.n_feats or int(np.sqrt(n_features))

    self.trees = []
    self.feat_idxs = []

    for _ in range(self.n_trees):
      # bootstrapping
      idxs = np.random.choice(n_samples,n_samples, replace=True)
      X_sample, y_sample = X[idxs], y[idxs]

      # feature bagging
      feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

      tree = build_tree(X_sample[:, feat_idxs], y_sample, max_depth=self.max_depth)
      self.trees.append(tree)
      self.feat_idxs.append(feat_idxs)

  def predict(self,X):
    tree_preds = np.zeros((X.shape[0],self.n_trees), dtype=int)
    for i, (tree, feat_idxs) in enumerate(zip(self.trees,self.feat_idxs)):
      tree_preds[:,i] = predict_tree_batch(tree, X[:,feat_idxs])

    y_pred =[]
    for row in tree_preds:
      counts = np.bincount(row)
      y_pred.append(np.argmax(counts))

    return np.array(y_pred)




In [ ]:
X_train.shape, y_train.shape

((800, 20), (800,))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf = RandomForest(n_trees=100, max_depth=MAX_DEPTH)
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print("Random Forest Performance:")
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy: ", accuracy_score(y_test, y_test_pred))
print("\nTest Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

Random Forest Performance:
Train Accuracy: 0.71125
Test Accuracy:  0.705

Test Confusion Matrix:
 [[  0  59]
 [  0 141]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.70      1.00      0.83       141

    accuracy                           0.70       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.50      0.70      0.58       200



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train sklearn Random Forest
sk_rf = RandomForestClassifier(
    n_estimators=100,     # number of trees
    max_depth=5,          # same as your scratch model
    random_state=42
)
sk_rf.fit(X_train, y_train)

# Predictions
y_train_pred = sk_rf.predict(X_train)
y_test_pred = sk_rf.predict(X_test)

# Performance
print("Sklearn Random Forest Performance:")
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy: ", accuracy_score(y_test, y_test_pred))
print("\nTest Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


Sklearn Random Forest Performance:
Train Accuracy: 0.81125
Test Accuracy:  0.755

Test Confusion Matrix:
 [[ 13  46]
 [  3 138]]

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.22      0.35        59
           1       0.75      0.98      0.85       141

    accuracy                           0.76       200
   macro avg       0.78      0.60      0.60       200
weighted avg       0.77      0.76      0.70       200



## Why `sklearn.RandomForestClassifier` Performed Better

Even though we used the same number of trees and the same max depth,
the `sklearn` implementation still outperformed our from-scratch version.  
This mainly comes down to:

1. **Bootstrap Sampling**  
   - Sklearn samples the training data *with replacement* for each tree.  
   - This ensures diversity between trees and reduces correlation.  
   - Our manual version likely skipped or simplified this step.

2. **Tree Quality**  
   - Sklearn grows trees more effectively, often until leaves are pure
     (with stopping rules like `min_samples_split`, `min_impurity_decrease`).  
   - Shallow or less optimal trees reduce the power of the ensemble.

3. **Optimisation Details**  
   - Sklearn uses efficient algorithms for finding the best splits.  
   - It handles categorical variables and thresholds more cleanly.  
   - It is highly optimised in C under the hood, giving more robust results.

\